# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import joblib
import numpy as np
import pandas as pd
import pkg_resources
import azureml.core
import pkg_resources
import xgboost


from matplotlib import pyplot as plt
from sklearn import datasets
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.57.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-268649
aml-quickstarts-268649
westeurope
610d6e37-4747-4a20-80eb-3aad70a55f43


In [3]:
experiment_name = 'CapstoneExp-AutoML'
project_folder = './Capstone-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
CapstoneExp-AutoML,quick-starts-ws-268649,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
found = False
key = "divorce-dataset"
description_text = "Split or Stay: Divorce Predictor Dataset - from kaggle"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
    
if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/duclinh/Capstone-Azure-Machine-Learning/refs/heads/main/Datasets/Divorce-Predictor-Dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()


{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,Atr1,Atr2,Atr3,Atr4,Atr5,Atr6,Atr7,Atr8,Atr9,Atr10,...,Atr46,Atr47,Atr48,Atr49,Atr50,Atr51,Atr52,Atr53,Atr54,Class
count,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,...,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000
mean,1.776471,1.652941,1.764706,1.482353,1.541176,0.747059,0.494118,1.452941,1.458824,1.576471,...,2.552941,2.270588,2.741176,2.382353,2.429412,2.476471,2.517647,2.241176,2.011765,0.494118
std,1.627257,1.468654,1.415444,1.504327,1.632169,0.904046,0.898698,1.546371,1.557976,1.421529,...,1.371786,1.586841,1.137348,1.511587,1.405090,1.260238,1.476537,1.505634,1.667611,0.501442
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,0.000000,0.000000
50%,2.000000,2.000000,2.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,2.000000,...,3.000000,2.000000,3.000000,3.000000,2.000000,3.000000,3.000000,2.000000,2.000000,0.000000
75%,3.000000,3.000000,3.000000,3.000000,3.000000,1.000000,1.000000,3.000000,3.000000,3.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.000000
max,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.000000


In [5]:
dataset.take(5).to_pandas_dataframe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,Atr1,Atr2,Atr3,Atr4,Atr5,Atr6,Atr7,Atr8,Atr9,Atr10,...,Atr46,Atr47,Atr48,Atr49,Atr50,Atr51,Atr52,Atr53,Atr54,Class
0,2,2,4,1,0,0,0,0,0,0,...,2,1,3,3,3,2,3,2,1,1
1,4,4,4,4,4,0,0,4,4,4,...,2,2,3,4,4,4,4,2,2,1
2,2,2,2,2,1,3,2,1,1,2,...,3,2,3,1,1,1,2,2,2,1
3,3,2,3,2,3,3,3,3,3,3,...,2,2,3,3,3,3,2,2,2,1
4,2,2,1,1,1,1,0,0,0,0,...,2,1,2,3,2,2,2,1,0,1


In [6]:
amlcompute_cluster_name = "capstone-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2',
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded...............................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# TODO: Put your automl settings here
automl_settings = {
                    "n_cross_validations": 5,
                    "primary_metric": 'accuracy',
                    # "primary_metric" : 'AUC_weighted'
                    "enable_early_stopping": True,
                    "max_concurrent_iterations": 4,
                    "experiment_timeout_minutes": 20,
                    "verbosity": logging.INFO
            }

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                    compute_target = compute_target,
                    task='classification',
                    training_data=dataset,
                    label_column_name='Class',
                    path = project_folder,
                    featurization= 'auto',
                    debug_log = "automl_errors.log",
                    **automl_settings
)

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output= True)

Submitting remote run.
No run_configuration provided, running on capstone-cluster with default configuration
Running on remote compute: capstone-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
CapstoneExp-AutoML,AutoML_d1e7eceb-75ec-4f91-b181-6cad43a96fb7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFea

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()
remote_run.wait_for_completion()

2024-10-14 06:41:16.523906: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-14 06:41:17.340855: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-14 06:41:17.580757: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-14 06:41:19.405616: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-14 06:41:23.648017: W tensorflow/comp

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_d1e7eceb-75ec-4f91-b181-6cad43a96fb7',
 'target': 'capstone-cluster',
 'status': 'Completed',
 'startTimeUtc': '2024-10-14T05:56:20.920547Z',
 'endTimeUtc': '2024-10-14T06:34:49.316127Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'capstone-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"CapstoneExp-AutoML","subscription_id":"610d6e37-4747-4a20-80eb-3aad70a55f43","resource_group":"aml-quickstarts-268649","workspace_name":"quick-starts-ws-268649","region":"westeurope","compute_target":"capst

In [10]:
for children_run in remote_run.get_children():
    print('------')
    print(children_run)

------
Run(Experiment: CapstoneExp-AutoML,
Id: AutoML_d1e7eceb-75ec-4f91-b181-6cad43a96fb7_42,
Type: azureml.scriptrun,
Status: Completed)
------
Run(Experiment: CapstoneExp-AutoML,
Id: AutoML_d1e7eceb-75ec-4f91-b181-6cad43a96fb7_43,
Type: azureml.scriptrun,
Status: Completed)
------
Run(Experiment: CapstoneExp-AutoML,
Id: AutoML_d1e7eceb-75ec-4f91-b181-6cad43a96fb7_41,
Type: azureml.scriptrun,
Status: Canceled)
------
Run(Experiment: CapstoneExp-AutoML,
Id: AutoML_d1e7eceb-75ec-4f91-b181-6cad43a96fb7_40,
Type: azureml.scriptrun,
Status: Canceled)
------
Run(Experiment: CapstoneExp-AutoML,
Id: AutoML_d1e7eceb-75ec-4f91-b181-6cad43a96fb7_39,
Type: azureml.scriptrun,
Status: Canceled)
------
Run(Experiment: CapstoneExp-AutoML,
Id: AutoML_d1e7eceb-75ec-4f91-b181-6cad43a96fb7_38,
Type: azureml.scriptrun,
Status: Completed)
------
Run(Experiment: CapstoneExp-AutoML,
Id: AutoML_d1e7eceb-75ec-4f91-b181-6cad43a96fb7_37,
Type: azureml.scriptrun,
Status: Completed)
------
Run(Experiment: Capston

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
# Get best run and model
best_run, fitted_model = remote_run.get_output()

# Print the best run
print(best_run)

Run(Experiment: CapstoneExp-AutoML,
Id: AutoML_d1e7eceb-75ec-4f91-b181-6cad43a96fb7_25,
Type: azureml.scriptrun,
Status: Completed)


In [12]:
best_run.get_metrics()

{'log_loss': 0.12389069119978206,
 'f1_score_macro': 0.9881944444444445,
 'average_precision_score_micro': 0.9993370681605975,
 'f1_score_weighted': 0.9881944444444445,
 'balanced_accuracy': 0.9882352941176471,
 'recall_score_macro': 0.9882352941176471,
 'weighted_accuracy': 0.9882352941176471,
 'average_precision_score_weighted': 1.0,
 'recall_score_weighted': 0.9882352941176471,
 'average_precision_score_macro': 1.0,
 'AUC_macro': 1.0,
 'accuracy': 0.9882352941176471,
 'precision_score_micro': 0.9882352941176471,
 'norm_macro_recall': 0.9764705882352942,
 'matthews_correlation': 0.9777046633277277,
 'AUC_micro': 0.9993079584775086,
 'AUC_weighted': 1.0,
 'f1_score_micro': 0.9882352941176471,
 'precision_score_weighted': 0.9894736842105264,
 'precision_score_macro': 0.9894736842105264,
 'recall_score_micro': 0.9882352941176471,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_d1e7eceb-75ec-4f91-b181-6cad43a96fb7_25/accuracy_table',
 'confusion_matrix': 'aml://artifactId/

In [13]:
best_run.get_details()

{'runId': 'AutoML_d1e7eceb-75ec-4f91-b181-6cad43a96fb7_25',
 'target': 'capstone-cluster',
 'status': 'Completed',
 'startTimeUtc': '2024-10-14T06:23:05.521851Z',
 'endTimeUtc': '2024-10-14T06:23:32.926156Z',
 'services': {},
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '5d5aa17e781d089e4f5dc0e6f1e065321541a74a',
  'pipeline_spec': '{"objects": [{"class_name": "MaxAbsScaler", "module": "sklearn.preprocessing", "param_args": [], "param_kwargs": {}, "prepared_kwargs": {}, "spec_class": "preproc"}, {"class_name": "ExtraTreesClassifier", "module": "sklearn.ensemble", "param_args": [], "param_kwargs": {"bootstrap": true, "class_weight": null, "criterion": "gini", "max_features": 0.3, "min_samples_leaf": 0.1131578947368421, "min_samples_split": 0.5252631578947369, "n_estimators": 50, "oob_score": false}, "prepared_kwargs": {}, "spec_class": "sklearn"}], "pipeline_id": "5d5aa17e781d089e4f5dc0e6f1e065321541a74a", "module": "sklearn.pipeline", "class_name": "Pipeline", "pipel

In [14]:
# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()

In [15]:
#TODO: Save the best model
myModel = best_run.register_model(model_path='outputs/model.pkl', model_name='capstoneModel_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'Accuracy': best_run_metrics['accuracy']})

print(myModel)

Model(workspace=Workspace.create(name='quick-starts-ws-268649', subscription_id='610d6e37-4747-4a20-80eb-3aad70a55f43', resource_group='aml-quickstarts-268649'), name=capstoneModel_automl, id=capstoneModel_automl:1, version=1, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '0.9882352941176471'})


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [22]:
environment = best_run.get_environment()
entry_script='inference/automl_scoring.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)


inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "automlaicservice", [myModel], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-10-14 06:59:03+00:00 Creating Container Registry if not exists..
2024-10-14 07:09:04+00:00 Registering the environment.
2024-10-14 07:09:06+00:00 Generating deployment configuration.
2024-10-14 07:09:08+00:00 Submitting deployment to compute..
2024-10-14 07:09:15+00:00 Checking the status of deployment automlaicservice..
2024-10-14 07:14:19+00:00 Checking the status of inference endpoint automlaicservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [23]:
print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Healthy
http://7d7bcd06-635b-4d8b-89be-e5a52219a7db.westeurope.azurecontainer.io/score
http://7d7bcd06-635b-4d8b-89be-e5a52219a7db.westeurope.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [24]:
# %run endpoint.py
import json

#Import test data
test_df = df.sample(5) # dataset is the pandas dataframe of the original data
label_df = test_df.pop('Class')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)

{"data": [{"Atr1": 0, "Atr2": 0, "Atr3": 1, "Atr4": 0, "Atr5": 0, "Atr6": 1, "Atr7": 0, "Atr8": 0, "Atr9": 0, "Atr10": 0, "Atr11": 0, "Atr12": 0, "Atr13": 0, "Atr14": 0, "Atr15": 0, "Atr16": 0, "Atr17": 0, "Atr18": 0, "Atr19": 0, "Atr20": 0, "Atr21": 0, "Atr22": 0, "Atr23": 0, "Atr24": 2, "Atr25": 0, "Atr26": 0, "Atr27": 0, "Atr28": 0, "Atr29": 0, "Atr30": 1, "Atr31": 0, "Atr32": 2, "Atr33": 0, "Atr34": 1, "Atr35": 0, "Atr36": 0, "Atr37": 0, "Atr38": 0, "Atr39": 0, "Atr40": 0, "Atr41": 0, "Atr42": 0, "Atr43": 3, "Atr44": 0, "Atr45": 0, "Atr46": 2, "Atr47": 2, "Atr48": 0, "Atr49": 0, "Atr50": 1, "Atr51": 1, "Atr52": 3, "Atr53": 4, "Atr54": 0}, {"Atr1": 4, "Atr2": 2, "Atr3": 3, "Atr4": 2, "Atr5": 4, "Atr6": 2, "Atr7": 0, "Atr8": 2, "Atr9": 3, "Atr10": 2, "Atr11": 4, "Atr12": 2, "Atr13": 4, "Atr14": 2, "Atr15": 3, "Atr16": 2, "Atr17": 4, "Atr18": 2, "Atr19": 4, "Atr20": 2, "Atr21": 3, "Atr22": 2, "Atr23": 4, "Atr24": 2, "Atr25": 4, "Atr26": 2, "Atr27": 3, "Atr28": 2, "Atr29": 4, "Atr30": 

In [25]:
import requests # Used for http post request

key = 'e00sdzt8FPdz2cEUCCykFhTHy8bR7Tgj'
# Set the content type
headers = {'Content-type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'
response = requests.post(service.scoring_uri, test_sample, headers=headers)

In [26]:
# Print results from the inference
print(response.text)

"{\"result\": [0, 1, 0, 1, 1]}"


In [27]:
# Print original labels
print(label_df)

101    0
41     1
87     0
42     1
21     1
Name: Class, dtype: int64


TODO: In the cell below, print the logs of the web service and delete the service

In [28]:
print(service.get_logs())

2024-10-14T07:14:05,201413000+00:00 - rsyslog/run 
2024-10-14T07:14:05,210357400+00:00 - gunicorn/run 
2024-10-14T07:14:05,213021500+00:00 | gunicorn/run | 
2024-10-14T07:14:05,219002000+00:00 | gunicorn/run | ###############################################
2024-10-14T07:14:05,224160000+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-10-14T07:14:05,229373000+00:00 | gunicorn/run | ###############################################
2024-10-14T07:14:05,234519500+00:00 | gunicorn/run | 
2024-10-14T07:14:05,243580000+00:00 - nginx/run 
2024-10-14T07:14:05,247043700+00:00 | gunicorn/run | 
2024-10-14T07:14:05,260611400+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20240918.v1
2024-10-14T07:14:05,262625100+00:00 | gunicorn/run | 
2024-10-14T07:14:05,270392200+00:00 | gunicorn/run | 
2024-10-14T07:14:05,277573400+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

In [ ]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
